In [1]:
#imports
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,BatchNormalization

Using TensorFlow backend.


In [7]:
#making model
model=Sequential()
inputShape=(28,28,1)
#Conv2D did not accept(784,-1),(784,testcases) as an input shape
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=inputShape))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=inputShape))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(10,activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

In [12]:
#making dataloader,
# this was mainly to convert pandas dataframe to numpy array
# keras inbuilt functionality and reshape functionality are
# available for numpy.

import pandas as pd
def MnistLoadData(X,Y):
    lenx=len(X)
    lenY=len(Y)
    #splitting input data to test and train
    x1=X.loc[0:lenx/2]
    x2=X.loc[lenx/2:lenx]
    y1=Y.loc[0:lenY/2]
    y2=Y.loc[lenY/2:lenY]
    #converting pandas datafram to numpy
    #for performing reshape and using 
    #to_categorical functions of keras
    x1=x1.to_numpy()
    x2=x2.to_numpy()
    y1=y1.to_numpy()
    y2=y2.to_numpy()
    x1=x1.reshape(x1.shape[0],28,28,1)
    x2=x2.reshape(x2.shape[0],28,28,1)
    #one hot encoding functionality of keras
    #converts y to its corresponding probablity array
    #1=[1,0,0,0,0,0,0,0...]
    #2=[0,1,0,0,0,0,0,....]
    y1=keras.utils.to_categorical(y1,10)
    y2=keras.utils.to_categorical(y2,10)
    return (x1,y1),(x2,y2)

#reading dataset
dataset=pd.read_csv('dataset/digit-recognizer/train.csv')
X=dataset.drop('label',axis=1)
Y=dataset['label']

#loading data
(x1,y1),(x2,y2)=MnistLoadData(X,Y)
#making model, here neural net extracts characteristic weights for this dataset
#so that it can make predictions
model.fit(x1,y1,batch_size=128,epochs=7,verbose=1,validation_data=(x2,y2))

Train on 21001 samples, validate on 21000 samples
Epoch 1/7
21001/21001 [==============================] - 29s 1ms/step - loss: 0.0015 - acc: 0.9999 - val_loss: 0.0595 - val_acc: 0.9851
Epoch 2/7
21001/21001 [==============================] - 33s 2ms/step - loss: 0.0016 - acc: 0.9999 - val_loss: 0.1005 - val_acc: 0.9761
Epoch 3/7
21001/21001 [==============================] - 34s 2ms/step - loss: 0.0016 - acc: 1.0000 - val_loss: 0.0676 - val_acc: 0.9833
Epoch 4/7
21001/21001 [==============================] - 36s 2ms/step - loss: 0.0014 - acc: 0.9998 - val_loss: 0.0721 - val_acc: 0.9825
Epoch 5/7
21001/21001 [==============================] - 34s 2ms/step - loss: 0.0012 - acc: 1.0000 - val_loss: 0.0683 - val_acc: 0.9838
Epoch 6/7
21001/21001 [==============================] - 34s 2ms/step - loss: 0.0012 - acc: 1.0000 - val_loss: 0.0619 - val_acc: 0.9844
Epoch 7/7
21001/21001 [==============================] - 30s 1ms/step - loss: 0.0012 - acc: 1.0000 - val_loss: 0.0606 - val_acc: 0.985

In [13]:
#check using the second half of the dataset, how accurate the prediction is
acc=model.evaluate(x2,y2,verbose=1)
print(acc)
#loading data from csv for predictions
#xtest need not be seperated since it only has input vector
Xtest=pd.read_csv('dataset/digit-recognizer/test.csv')
Xtest=Xtest.to_numpy()
Xtest=Xtest.reshape(Xtest.shape[0],28,28,1)
Ytest=model.predict(Xtest)

21000/21000 [==============================] - 7s 333us/step
[0.060601188722526005, 0.9850476190476191]


In [14]:
#making the submission file
from numpy import argmax
import numpy as np
Yarray=[]
i=1
for nu in Ytest:
    Yarray.append((i,int(argmax(nu))))
    i=i+1

np.savetxt("dataset/digit-recognizer/submission.csv",np.asarray(Yarray),delimiter=",",fmt="%d")